# Python data form
1.1. card example
   * __len__
   * __getitem__ : 
      ** literable >>> for
      ** slicing >>> \[\]
   * __contains__  >>> in
   * sorted(list,key)
   * __setitem__ 洗牌 >>> chapter 11


In [8]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]
    
# for example 
# 1.很好的利用的python 的其他方法 2.不用记住。
deck = FrenchDeck()
print(len(deck))
from random import choice
print(choice(deck))
# slicing 
deck[:3] # top 3
deck[12::13] # 13
    # literable
for card in reversed(deck[:3]):
    print(card)


52
Card(rank='2', suit='clubs')
Card(rank='4', suit='spades')
Card(rank='3', suit='spades')
Card(rank='2', suit='spades')


1.2 使用特殊方法
<br>然而如果是 Python 内置的类型,比如列表(list)、字符串(str)、
字节序列(bytearray)等,那么 CPython 会抄个近路,__len__ 实际
上会直接返回 PyVarObject 里的 ob_size 属性。PyVarObject 是表示
内存中长度可变的内置对象的 C 语言结构体。直接读取这个值比调用一
个方法要快很多。<br>
1.2.1模拟数值类型

In [9]:
from math import hypot

class Vector:

    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def __repr__(self):
        # 字符串表示形式的 >>> print
        # difference about repr and str
        # repr : for developer , to be unambiougous
        # str:  for user , to be clear to read
        # if no str, python will find str
        return 'Vector(%r, %r)' % (self.x, self.y)

    def __abs__(self):
        return hypot(self.x, self.y)

    def __bool__(self):
        """
        bool(x) 的背后是调用x.__bool__() 的结果;
        如果不存在 __bool__ 方法,那么 bool(x) 会尝试调用 x.__len__()。
        若返回 0,则 bool 会返回 False;否则返回True
        """
        return bool(abs(self))
    #   return bool(self.x or self.y)

    def __add__(self, other):
        #代码里只是读取了它们的值而已。中缀运算符的基本原则就是不改变操作对象,而
        #是产出一个新的值。第 13 章会谈到更多这方面的问题。
        x = self.x + other.x
        y = self.y + other.y
        return Vector(x, y)

    def __mul__(self, scalar):
        # 交换律则被忽略了>>> 13 __rmul__
        return Vector(self.x * scalar, self.y * scalar)

# Arrry and sequence

2.1 内置序列类型概览

__容器序列__
   * list、tuple 和 collections.deque 这些序列能存放不同类型的数据。
   * by reference
   
__扁平序列__
   * str、bytes、bytearray、memoryview 和 array.array,这类序列只能容纳一种类型。
   * by value >>> 连续的内存空间，所以更集凑。
   
__可变序列__
* list、bytearray、array.array、collections.deque 和 memoryview。
  
__不可变序列__
* tuple、str 和 bytes

![diffence of mutable](img/2_1.png)


2.2 列表推导和生成器表达式 list comprehension generator expression

2.2.1 __listcomps__
    
    通常的原则是,只用列表推导来创建新的列表,并且尽量保持简短。如果列表推导的代码超过了两行,你可能就要考虑是不是得用 for 循环重写了


In [12]:
symbols = '$¢£¥€¤'
codes = [ord(symbol) for symbol in symbols]
codes

[36, 162, 163, 165, 8364, 164]

2.2.2 __listcomps__ and __filter map__

filter 和 map 合起来能做的事情,列表推导也可以做,而且还不需要
借助难以理解和阅读的 lambda 表达式. >>> chapter 5

In [17]:
symbols = '$¢£¥€¤'
beyond_ascii_1 = [ord(x) for x in symbols if ord(x) >127]
beyond_ascii_2 = list(filter(lambda c: c> 127, map(ord,symbols)))
beyond_ascii_1 == beyond_ascii_2

True

In [18]:
import timeit

TIMES = 10000

SETUP = """
symbols = '$¢£¥€¤'
def non_ascii(c):
    return c > 127
"""

def clock(label, cmd):
    res = timeit.repeat(cmd, setup=SETUP, number=TIMES)
    print(label, *('{:.3f}'.format(x) for x in res))

clock('listcomp        :', '[ord(s) for s in symbols if ord(s) > 127]')
clock('listcomp + func :', '[ord(s) for s in symbols if non_ascii(ord(s))]')
clock('filter + lambda :', 'list(filter(lambda c: c > 127, map(ord, symbols)))')
clock('filter + func   :', 'list(filter(non_ascii, map(ord, symbols)))')

listcomp        : 0.008 0.007 0.007 0.007 0.009
listcomp + func : 0.012 0.012 0.012 0.011 0.011
filter + lambda : 0.012 0.012 0.012 0.012 0.012
filter + func   : 0.011 0.011 0.011 0.011 0.011


2.2.3 笛卡儿积
__!!__ :

成器表达式背后遵守了迭代器协议,可以逐个地产出元素,而不是先建立一个完整的列表,然后再把这个列表传递到某个构造函数里。前面那种方式显然能够节省内存. 

生成器表达式的语法跟列表推导差不多,只不过把方括号换成圆括号而已 >> 14 chapter


In [19]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
tshirts = [(color,size) for color in colors for size in sizes]
# order matter


2.3 元组不仅仅是不可变的列表
* 有字段名的记录
<br>元组其实是对数据的记录:元组中的每个元素都存放了记录中一个字段
的数据,外加这个字段的位置。正是这个位置信息给数据赋予了意义
* 不可变的列表

In [28]:
#2.3.2 packing and unpacking 
a =1
b =2
b, a = a, b
#还可以用 * 运算符把一个可迭代对象拆开作为函数的参数:
def myadd(x,y):
    return x+y
t = (1,1)
print(myadd(*t))
import os
_, filename = os.path.split('/home/luciano/.ssh/idrsa.pub')
print(filename)
# *args 处理剩下的元素, any position
a,b,*rest = range(5)
print(a,b,rest)
a, *body, c, d = range(5)
print(a, body, c, d)
# 可以是嵌套的
#for name, cc, pop, (latitude, longitude) in metro_areas:

2
idrsa.pub
0 1 [2, 3, 4]
0 [1, 2] 3 4


2.3.4具名元组 

    我们时常会需要给记录中的字段命名
    collections.namedtuple 是一个工厂函数,它可以用来构建一个带字段名的元组和一个有名字的类——这个带名字的类对调试程序有很大帮助。
    

2.4 切片
2.4.1 切片和区间会忽略最后一个元素
 * range(3) 和 my_list[:3] 都返回 3 个元素
 * 快速计算出切片和区间的长度 stop - start
 * my_list[:x] 和 my_list[x:]
2.4.2 多维切片和省略
seq[start:stop:step] 进行求值的时候,Python 会调用seq.__getitem__(slice(start, stop, step))
2.4.4 给切片赋值


In [31]:
l = list(range(10))
print(l)
l[2:5] = [20, 30]
print(l)
del l[5:7]
print(l)
l[3::2] = [11, 22]
print(l)
try:
    l[2:5] = 100
    print("error")
except:
    l[2:5] = [100]
print(l)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 20, 30, 5, 6, 7, 8, 9]
[0, 1, 20, 30, 5, 8, 9]
[0, 1, 20, 11, 5, 22, 9]
[0, 1, 100, 22, 9]


2.5 对序列使用+和*
+ 和 * 都遵循这个规律,不修改原有的操作对象,而是构建一个全新的序列

In [35]:
l = [1, 2, 3]
l * 5
5 * 'abcd'

'abcdabcdabcdabcdabcd'

In [36]:
board = [['_'] * 3 for i in range(3)] # do 3 times
board[1][2] = 'X'
print(board)

weird_board = [['_'] * 3] * 3 # reference 
weird_board[1][2] = 'O'
print(weird_board)

[['_', '_', '_'], ['_', '_', 'X'], ['_', '_', '_']]
[['_', '_', 'O'], ['_', '_', 'O'], ['_', '_', 'O']]


In [38]:
row=['_'] * 3
board = []
for i in range(3):
    board.append(row)
   
board_2 = []
for i in range(3):
    row=['_'] * 3
    board.append(row)

2.6 序列的增量赋值 += *=
 
 如果 a 实现了 __iadd__ 方法,就会调用这个方法。同时对可变序列
(例如 list、bytearray 和 array.array)来说,a 会就地改动,就
像调用了 a.extend(b) 一样。但是如果 a 没有实现 __iadd__ 的话,a
+= b 这个表达式的效果就变得跟 a = a + b 一样了:首先计算 a +
b,得到一个新的对象,然后赋值给 a。也就是说,在这个表达式中,
变量名会不会被关联到新的对象,完全取决于这个类型有没有实现
__iadd__ 这个方法

2.7 list.sort方法和内置函数sorted

list.sort 方法会就地排序列表,不会把原列表复制一份。返回值是 None,
在这种情况下返回 None 其实是 Python 的一个惯例:
例如,random.shuffle 函数也遵守了这个惯例。


2.8 用bisect来管理已排序的序列
bisect(haystack, needle) 在 haystack(干草垛)里搜索needle(针)的位置,该位置满足的条件是,把 needle 插入这个位置之后,haystack 还能保持升序
* bisect() find position and insert()
* insort() finish one step >>> faster

In [1]:
import bisect
import sys

HAYSTACK = [1, 4, 5, 6, 8, 12, 15, 20, 21, 23, 23, 26, 29, 30]
NEEDLES = [0, 1, 2, 5, 8, 10, 22, 23, 29, 30, 31]

ROW_FMT = '{0:2d} @ {1:2d}    {2}{0:<2d}'

def demo(bisect_fn):
    for needle in reversed(NEEDLES):
        position = bisect_fn(HAYSTACK, needle)  # <1>
        offset = position * '  |'  # <2>
        print(ROW_FMT.format(needle, position, offset))  # <3>

if __name__ == '__main__':

    if sys.argv[-1] == 'left':    # <4>
        bisect_fn = bisect.bisect_left
    else:
        bisect_fn = bisect.bisect

    print('DEMO:', bisect_fn.__name__)  # <5>
    print('haystack ->', ' '.join('%2d' % n for n in HAYSTACK))
    demo(bisect_fn)

# END BISECT_DEMO

DEMO: bisect_right
haystack ->  1  4  5  6  8 12 15 20 21 23 23 26 29 30
31 @ 14      |  |  |  |  |  |  |  |  |  |  |  |  |  |31
30 @ 14      |  |  |  |  |  |  |  |  |  |  |  |  |  |30
29 @ 13      |  |  |  |  |  |  |  |  |  |  |  |  |29
23 @ 11      |  |  |  |  |  |  |  |  |  |  |23
22 @  9      |  |  |  |  |  |  |  |  |22
10 @  5      |  |  |  |  |10
 8 @  5      |  |  |  |  |8 
 5 @  3      |  |  |5 
 2 @  1      |2 
 1 @  1      |1 
 0 @  0    0 


In [2]:
# bisect 可以用来建立一个用数字作为索引的查询表格 excel int >>> string
def grade(score, breakpoints=[60, 70, 80, 90], grades='FDCBA'):
    i = bisect.bisect(breakpoints, score)
    return grades[i]
[grade(score) for score in [33, 99, 77, 70, 89, 90, 100]]

['F', 'A', 'C', 'C', 'B', 'A', 'A']

In [4]:
import bisect
import random
SIZE=7

random.seed(1729)
my_list = []
for i in range(SIZE):
    new_item = random.randrange(SIZE*2)
    bisect.insort(my_list, new_item)
    print('%2d ->' % new_item, my_list)

10 -> [10]
 0 -> [0, 10]
 6 -> [0, 6, 10]
 8 -> [0, 6, 8, 10]
 7 -> [0, 6, 7, 8, 10]
 2 -> [0, 2, 6, 7, 8, 10]
10 -> [0, 2, 6, 7, 8, 10, 10]


2.9 当列表不是首选时
* float >> array
* fluently first in first out >>> deque
* unique >>> set because set is unorder


2.9.1 array 
数组还提供从文件读取和存入文件的更快的方法,如.frombytes 和 .tofile

python array is simlar with C eg: array('b') 

a = array.array(a.typecode, sorted(a))



2.9.2 双向队列和其他形式的队列
<br>
append + pop >>> used as stack last in first out
<br>addend + pop(0) >>> first in first out quequen >>> time-consuming 

collections.deque 类(双向队列)是一个线程安全、可以快速从两
端添加或者删除元素的数据类型。


__queue__
<br>
__PriorityQueue, Queue、LifoQueue__

不同的线程可以利用这些数据类型来交换信息。这三个类的构造方法都有一个可选参maxsize,它接收正整数作为输入值,用来限定队列的大小。
但是在满员的时候,这些类不会扔掉旧的元素来腾出位置。
如果队列满了,它就会被锁住,直到另外的线程移除了某个元素而腾出了位置.
这一特性让这些类很适合用来控制活跃线程的数量

__multiprocessing__ :
它跟 queue.Queue 类似,是设计给进程间通信用的

In [9]:
from collections import deque
dq = deque(range(10), maxlen=10)
print(dq)
dq.rotate(3)
print(dq)
dq.rotate(-4)
"""
当 n > 0 时,队列的最右边的 n个元素会被移动到队列的左边。
当 n < 0 时,最左边的 n 个元素会被移动到右边
"""
print(dq)
dq.appendleft(-1) #已满队列做尾部添加操作的时候,它头部的元素会被删除掉 
print(dq)
dq.extendleft([10, 20, 30, 40])
"""
extendleft(iter) 方法会把迭代器里的元素逐个添加到双向队列
的左边,因此迭代器里的元素会逆序出现在队列里。
"""
print(dq)


deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], maxlen=10)
deque([7, 8, 9, 0, 1, 2, 3, 4, 5, 6], maxlen=10)
deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 0], maxlen=10)
deque([-1, 1, 2, 3, 4, 5, 6, 7, 8, 9], maxlen=10)
deque([40, 30, 20, 10, -1, 1, 2, 3, 4, 5], maxlen=10)


# dictionary and set

* 散列表则是字典类型性能出众的根本原因 集合(set)的实现其实也依赖于散列表<br>
什么是可散列数据类型 hasable <br>
如果一个对象是可散列的，那么在这个对象的生命周期中，它的散列值是不变的，而且这个对象需要实现 __hash__() 方法。
另外可散列对象还要有 __qe__() 方法，这样才能跟其他键做比较

eg：  str bytes 和数值类型，forozenset >>> 只能容纳可散列的<br>
元组： 内含元素必须可散列


In [4]:
# 3.2 字典推导
DIAL_CODES = [ (86, 'China'), (91, 'India'), (1, 'United States'), (62, 'Indonesia'), (55, 'Brazil'), (92, 'Pakistan'), (880, 'Bangladesh'), (234, 'Nigeria'),
(7, 'Russia'), (81, 'Japan') ]

country = {country: code for code, country in DIAL_CODES}

country_select = {code: country.upper() for country, code in country.items() if code < 66}

NameError: name 'country_code' is not defined

# 7 
*  Python 如何计算装饰器句法
*  Python 如何判断变量是不是局部的
*  闭包存在的原因和工作原理
*  nonlocal 能解决什么问题
*  实现行为良好的装饰器
*  标准库中有用的装饰器
*  实现一个参数化装饰器

7.1 Basic knowledge 
<br>
装饰器是可调用的对象，其参数是另一个函数（被装饰的函数), 有时，这样做更方便，尤其是做元编程（在运行时改变程序的行为）时<br>
Feature: **能把被装饰的函数替换成其他函数,装饰器在加载模块时立即执行**

In [1]:
def deco(func):
    def inner():
        print("running inner")
    return inner
@deco
def target():
    print("running target")
target()

running inner


7.2 Python何时执行装饰器
被装饰的函数定义之后立即运行。通常是在导入时（即 Python 加载模块时)
<br>  >>>**导入时和运行时**之间的区别。
*  装饰器函数与被装饰的函数在同一个模块中定义。实际情况是，装饰器通常在一个模块中定义，然后应用到其他模块中的函数上。
*  register 装饰器返回的函数与通过参数传入的相同。实际上，大多数装饰器会在内部定义一个函数，然后将其返回。

In [2]:
registry = [] 
def register(func): 
    print('running register(%s)' % func) 
    registry.append(func) 
    return func 
@register 
def f1():
    print('running f1()')
@register
def f2():
    print('running f2()')
def f3(): 
    print('running f3()')
def main(): 
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
main()

running register(<function f1 at 0x00000243E26A0BF8>)
running register(<function f2 at 0x00000243E26A0E18>)
running main()
registry -> [<function f1 at 0x00000243E26A0BF8>, <function f2 at 0x00000243E26A0E18>]
running f1()
running f2()
running f3()


7.4变量作用域规则<br>
Python 编译函数的定义体时，它判断 b 是局部变量，因为在函数中给它赋值了。生成的字节码证实了这种判断

In [3]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9 # error
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

7.5 闭包<br>
difference 闭包和匿名函数<br>**闭包**指延伸了作用域的函数，其中包含函数定义体中引用、但是不在定义体中定义的非全局变量。
<br> 函数是不是匿名的没有关系，关键是它能访问定义体之外定义的非全局变量。
<br> Python 的变量访问: 本地没有，找上级。 <br>>>> 闭包是一种函数，它会保留定义函数时存在的自由变量的绑定，这样调用函数时，虽然定义作用域不可用了，但是仍能使用那些绑定
![close package](img/7_1.jpg)

In [ ]:
#avg history fucntion
# class method
class Averager(): # 实例是可调用对象
    def __init__(self):
        self.series = []
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)
def make_averager(): #调用 make_averager 时，返回一个 averager 函数对象。每次调用averager 时，它会把参数添加到系列值中，然后计算当前平均值
    series = []
        def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager


7.6 nonlocal<br>
作用：对数字、字符串、元组等不可变类型来说，只能读取，不能更新。如果尝试重新绑定,<br>其实会隐式创建局部变量 count。这样，count 就不是自由变量了，因此不会保存在闭包中

In [4]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        count += 1 #error !!local variable 'count' referenced before assignment
        total += new_value
        return total / count
    return averager

def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager

# 8 对象引用、可变性和垃圾回收
8.1变量不是盒子<br>
是便利贴<br>
8.2 标识、相等性和别名
8.2.1 
is 运算符比 == 速度快，因为它不能重载，所以 Python 不用寻找并调用特殊方法，而是直接比较两个整数 ID。<br>
而 a == b 是语法糖，等同于a.__eq__(b)。继承自 object 的 __eq__ 方法比较两个对象的 ID，结果与 is 一样。<br>
但是多数内置类型使用更有意义的方式覆盖了 __eq__ 方法，会考虑对象属性的值。相等性测试可能涉及大量处理工作，<br>
例如，比较大型集合或嵌套层级深的结构时。

In [7]:
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles #别名
print("is: ",lewis is charles)
print("id: ",id(charles), id(lewis))
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950} #冒充者
print("==",alex == charles) #比较两个对象，结果相等，这是因为 dict 类的 __eq__ 方法就是这样实现的
print("is not: " ,alex is not charles) #但它们是不同的对象。这是 Python 说明标识不同的方式：a is notb。

is:  True
id:  2490585300064 2490585300064
== False
is not:  True


8.5 del和垃圾回收<br>
del 语句删除名称，而不是对象。del 命令可能会导致对象被当作垃圾回收，<br>
但是仅当删除的变量保存的是对象的最后一个引用，或者无法得到对象时。 <br>
重新绑定也可能会导致对象的引用数量归零，导致对象被销毁。<br>
在 CPython 中，垃圾回收使用的主要算法是引用计数。实际上，每个对象都会统计有多少引用指向自己。<br>
当引用计数归零时，对象立即就被销毁：CPython 会在对象上调用 __del__ 方法（如果定义了），然后释放分配给对象的内存。<br>
CPython　2.0 增加了分代垃圾回收算法，用于检测引用循环中涉及的对象组,<br>
如果一组对象之间全是相互引用，即使再出色的引用方式也会导致组中的对象不可获取。<br>
Python 的其他实现有更复杂的垃圾回收程序，而且不依赖引用计数，<br>
这意味着，对象的引用数量为零时可能不会立即调用 __del__ 方法<br>

# 16
字典为动词“to yield”给出了两个释义：产出和让步。对于 Python 生成器
中的 yield 来说，这两个含义都成立。yield item 这行代码会产出一
个值，提供给 next(...) 的调用方；此外，还会作出让步，暂停执行
生成器，让调用方继续工作，直到需要使用另一个值时再调用
next()。调用方会从生成器中拉取值。